### Binary Classifier on a single label using text_tokens as content

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import twitter_preproc

conf = SparkConf().setAll([
    ("num-executors", 4), 
    ("total-executor-cores", 16), 
    ("executor-memory", "8g"),
    ("spark.yarn.executor.memoryOverhead", "64g")])
sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [3]:
datapath = "///tmp/traintweet_10k.tsv"
target = "like_timestamp"
user = "engaging_user_id"

Load DF and change Timestamp/None to 1/0 in target column

In [4]:
import importlib
importlib.reload(twitter_preproc)
preproc = twitter_preproc.twitter_preproc(spark, sc, datapath, method="CB")
df = preproc.getDF().withColumn(target.split("_")[0],when(df[target].isNotNull(),1)

DataFrame[text_tokens: string]

### Handle BERT tokens like words, maybe TODO: find deeper meaning in the tokens(embeddings)

In [5]:
from pyspark.ml.feature import RegexTokenizer,NGram,CountVectorizer,IDF,StringIndexer
from pyspark.ml import Pipeline

stringIndexer = StringIndexer(inputCol=user, outputCol=user+"_idx")
regexTokenizer = RegexTokenizer(inputCol="text_tokens", outputCol="terms", pattern="\t")
cv = CountVectorizer(inputCol="terms", outputCol="vector")
idf = IDF(inputCol="vector", outputCol="features")
pipeline = Pipeline(stages=[stringIndexer,regexTokenizer, cv,idf])

model = pipeline.fit(df)


In [ ]:
transformed = model.transform(df)

In [ ]:
transformed.select("features",user+_"idx").show(4)

In [ ]:
#sc.stop()